In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
#picaso
from picaso import justdoit as jdi
from picaso import justplotit as jpi
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

# How do I load in target properties from Exo.Mast? 

In [2]:
nexsci = jdi.all_planets()

In [3]:
nexsci.head()

,pl_name,pl_letter,hostname,hd_name,hip_name,tic_id,disc_pubdate,disc_year,discoverymethod,disc_locale,...,sy_pmstr,sy_pm_reflink,sy_pmra,sy_pmraerr1,sy_pmraerr2,sy_pmrastr,x,y,z,htm20
0,OGLE-2016-BLG-1227L b,b,OGLE-2016-BLG-1227L,NaN,NaN,NaN,2020-03,2020.0,Microlensing,Ground,...,NaN,<a refstr=HAN_ET_AL__2020 href=https://ui.adsa...,NaN,NaN,NaN,NaN,-0.063824,-0.828921,-0.555712,-7.684157e+08
1,Kepler-276 c,c,Kepler-276,NaN,NaN,TIC 138213510,2014-02,2013.0,Transit,Space,...,8.0702604&plusmn;0.0515101,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,7.99553,0.051581,-0.051581,7.9955300&plusmn;0.0515805,0.310575,-0.711954,0.629813,1.302658e+09
2,Kepler-829 b,b,Kepler-829,NaN,NaN,TIC 123451768,2016-05,2016.0,Transit,Space,...,8.3089089&plusmn;0.0412978,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,7.93777,0.040833,-0.040833,7.9377700&plusmn;0.0408326,0.157566,-0.720680,0.675124,-7.922609e+08
3,K2-283 b,b,K2-283,NaN,NaN,TIC 266017624,2018-12,2018.0,Transit,Space,...,11.1101761&plusmn;0.0531852,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,-10.97340,0.053624,-0.053624,-10.9734000&plusmn;0.0536244,0.959702,0.224997,0.168368,6.098136e+08
4,Kepler-477 b,b,Kepler-477,NaN,NaN,TIC 158633329,2016-05,2016.0,Transit,Space,...,26.7,<a refstr=LATHAM_ET_AL__2005 href=https://ui.a...,-26.00000,NaN,NaN,-26.0,0.229185,-0.702544,0.673726,-1.968935e+09


In [4]:
nexsci.loc[nexsci['hostname']=='HAT-P-26']

,pl_name,pl_letter,hostname,hd_name,hip_name,tic_id,disc_pubdate,disc_year,discoverymethod,disc_locale,...,sy_pmstr,sy_pm_reflink,sy_pmra,sy_pmraerr1,sy_pmraerr2,sy_pmrastr,x,y,z,htm20
711,HAT-P-26 b,b,HAT-P-26,NaN,NaN,TIC 420779000,2011-02,2010.0,Transit,Ground,...,147.8161704&plusmn;0.0835536,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,37.8389,0.093273,-0.093273,37.8389000&plusmn;0.0932728,-0.835079,-0.545556,0.070791,-440551644.0


In [5]:
#first isolate the row 
hatp26_row = nexsci.loc[nexsci['hostname']=='HAT-P-26']

#add it to load planet function with opacity
opa = jdi.opannection(wave_range=[1,5])
hatp26 = jdi.load_planet(hatp26_row,opa , st_metfe=0)#hatp26 is misst st_metfe so we must add it as a keyword

Load planet function adds in: 
    - planet properties and stellar properties 
    - parameterized PT profile 

That means you still need to add in the chemistry and cloud parameters

In [6]:
co=1.0#c/o ratio 1x solar
logfeh = 1.5 #10^1.5 ~ 30x solar 
hatp26.chemeq_visscher(co,logfeh) #adds in template chemistry from sonora
df = hatp26.spectrum(opa, calculation='transmission+thermal')

In [7]:
x,y = jdi.mean_regrid(df['wavenumber'], df['fpfs_thermal'],R=150)
plot = [jpi.spectrum(x,y,plot_width=400, y_axis_type='log',y_axis_label='FpFs')]
x,y = jdi.mean_regrid(df['wavenumber'], df['transit_depth'],R=150)
plot += [jpi.spectrum(x,y,plot_width=400,y_axis_label='(Rp/Rs)^2')]

jpi.show(jpi.row(plot))

# How do I access the pressure-temperature profile parameterizations?

In [8]:
#start by loading in some template properties
opa = jdi.opannection(wave_range=[1,5])
hatp26_row = nexsci.loc[nexsci['hostname']=='HAT-P-26']
hatp26 = jdi.load_planet(hatp26_row,opa, st_metfe=0)

### How do the input parameters effect the parameterization?

In [9]:
fig = [jpi.figure(width=300, height=300, y_axis_type='log', y_range=[100,1e-6],x_range=[400,1700],
                  y_axis_label='Pressure(bars)',x_axis_label='Temperature (K)',
                 title='Effect of logg1')]
colors = jpi.pals.magma(10)
for i, logg1 in enumerate(np.linspace(-2,2,10)):
    hatp26.guillot_pt(576.17, T_int=100, logg1=logg1, logKir=-1.5)
    fig[0].line(hatp26.inputs['atmosphere']['profile']['temperature'],
            hatp26.inputs['atmosphere']['profile']['pressure'],color=colors[i])

fig += [jpi.figure(width=300, height=300, y_axis_type='log', y_range=[100,1e-6],x_range=[400,1700], 
                 x_axis_label='Temperature (K)',title='Effect of LogKir')]
for i,logKir in enumerate(np.linspace(-2,2,10)):
    hatp26.guillot_pt(576.17, T_int=100, logg1=-2, logKir=logKir)
    fig[1].line(hatp26.inputs['atmosphere']['profile']['temperature'],
            hatp26.inputs['atmosphere']['profile']['pressure'], color=colors[i])
        
fig += [jpi.figure(width=300, height=300, y_axis_type='log', y_range=[100,1e-6],x_range=[400,1700], 
                 x_axis_label='Temperature (K)',title='Effect of Tint')]       
for i,T_int in enumerate(np.linspace(50,300,10)):
    hatp26.guillot_pt(576.17, T_int=T_int, logg1=-2, logKir=-2)
    fig[2].line(hatp26.inputs['atmosphere']['profile']['temperature'],
            hatp26.inputs['atmosphere']['profile']['pressure'], color=colors[i])

jpi.show(jpi.row(fig))
                

# How do I return the output from photon attenuation plot? 

In [10]:
#first isolate the row 
hatp26_row = nexsci.loc[nexsci['hostname']=='HAT-P-26']

#add it to load planet function with opacity
opa = jdi.opannection(wave_range=[0.3,1])
hatp26 = jdi.load_planet(hatp26_row,opa ,st_metfe=0)
co = 1
logfeh=2
hatp26.chemeq_visscher(co,logfeh)
df = hatp26.spectrum(opa, calculation='reflected',full_output=True)

In [11]:
plot,wave,at_pressures_gas,at_pressures_cld,at_pressures_ray = jpi.photon_attenuation(df['full_output'],
                                                                                     return_output=True)

In [12]:
#now you can make your own plot!
fig = jpi.figure(y_axis_type='log',y_range=[10,1e-3],height=250, width=500)
for i,iy,ilab in zip([0,1],[at_pressures_gas,at_pressures_ray ], ['Molecular','Rayleigh']):
    x,y = jpi.mean_regrid(1e4/wave,iy, R=150)
    fig.line(1e4/x,y,legend_label=ilab, line_width=4, color=jpi.Colorblind8[i])
jpi.show(fig)

# Can I use the phase angle function to specify a non-zero phase for thermal emission? 

The phase angle function computes the incoming and outgoing angles. However, non-zero phase functionality is specific to __reflected light observations only__. This might be confusing because of course it is possible to observe thermal emission of planet at non-zero phase. However, unlike reflected light, thermal emission radiates from the planet in all directions (regardless of phase angle). 

__Do not force the code to run at non-zero phase angles for thermal emission.__ 

# I'm confused about what opacity file to use

Question: I'm confused about the opacity files. Version 1.0 (opacity.db) is the low res version that covers the "useful" wavelengths (0.3 to 14 micron) while version 2.0 has two files that cover the 0.6-6 micron at higher resolution and 4.8-15 micron. What should I download? And once I download it, where do I put them? If it's just the one file that seems straightforward in terms of where to put it but what happens when there are two files (or all 3) in the same folder?

Answer: 

**What do I download?**

The [low sampled files across a large wavelength range](https://zenodo.org/record/3759675#.Y8HVkezMLvU), which is on Zenodo as V1 is great for quick calculations that don't necessarily need to be accurate. For example: proposals, example models, any testing, retrievals on fake data. 

However, when comparing to real data, it's important to use higher sampling. [This tutorial](https://natashabatalha.github.io/picaso/notebooks/10_ResamplingOpacities.html) shows users estimated sampling errors. Therefore, it is important to have higher sampling files as well. The [higher sampling files are located here under V2](https://zenodo.org/record/6928501#.Y8HWH-zMLvU).  

So depending on your use case, the answer might be: download both! 

**Where do I put all the files?**

`PICASO` uses the function [`justdoit.opannection`](https://natashabatalha.github.io/picaso/picaso.html#picaso.justdoit.opannection) to grab the opacity file located in the reference directory [`opacities`](https://github.com/natashabatalha/picaso/tree/master/reference/opacities). In the installation instructions you will notice there is a step to place the zenodo file here. Just for completeness, internally, we specify the name of this [file here](https://github.com/natashabatalha/picaso/blob/891343fcc41faa345f8b85aaa8d50c4939c421a3/reference/config.json#L91). 

The general recommendation is to keep one "default" file in your `reference/opacities` folder so that you do not need to worry about always specifying a file when running the code. Then assign one place, easy to locate, where you include the rest of the files. In order to access these will need to point to this file path  using the `filename_db` keyword in `opannection`. 


# Has your question not been answered? Feel free to contact us! 

Submit an issue on Github: https://github.com/natashabatalha/picaso/issues